In [1]:
import os
from datetime import datetime
import pickle
import numpy as np
import tqdm
import json
from datetime import datetime
progress_bar = True
import time
import dynetx as dn
import openMindednessFunctions as omf

# Estimating open mindedness distributions + other info

In [2]:
def createDataset(t0, tN):
    data = {}
    for dataset_name in ['euro2020']:
        '''
        per ogni time step mi ritorna un dizionario nodo: {dizionario con delle info}
        '''
        data[dataset_name] = {}
        timestamps = {0: 't0', 1: 't1'}
        t2node2opinions = omf.create_dictionaries(timestamps, dataset_name)
        for t in tqdm.tqdm(range(t0, tN)):
            data[dataset_name][t] = {}
            t1=t+1
            g = omf.createGraph(dataset_name, timestamps, t)
            for v in tqdm.tqdm(g.nodes()):
                if v in t2node2opinions[t1].keys():
                    opvt = t2node2opinions[t][v]
                    opvt1 = t2node2opinions[t1][v]
                    neighs = list(g.neighbors(v))
                    sortedNeighOps = omf.sortNeighOps(g, v, t, t2node2opinions, opvt)
                    eps, err, estOp = omf.estimation(opvt, opvt1, sortedNeighOps)
                    orientation = omf.politicalLeaning(opvt) 
                    nactiveint = len([op for op in sortedNeighOps if op <= eps])
                    data[dataset_name][t][v] = {'dataset':dataset_name, 'opt':opvt, 'opt1': opvt1, 'neighbors':neighs, 
                                                    'opChange':opvt1-opvt, 'orientation':orientation, 
                                                    'neighborsOps':sortedNeighOps, 'error':err, 'estOp':estOp, 
                                                    'openMindedness':eps, 'nactiveint':nactiveint}    
                    if t > t0:
                        try:
                            data[dataset_name][t][v]['orientation-1'] = data[t-1][v]['orientation']
                        except:
                            data[dataset_name][t][v]['orientation-1'] = None
    return data

In [3]:
data = createDataset(0, 1)
with open(f'openMindednessData.pickle', 'wb') as ofile:
    pickle.dump(data, ofile)

100%|██████████| 1/1 [00:00<00:00, 17.91it/s]


In [4]:
data

{'euro2020': {0: {'775a7d495a555852090c': {'dataset': 'euro2020',
    'opt': 0.3333333333333333,
    'opt1': 0.8333333333333334,
    'neighbors': ['5554555054555d5449565e',
     '5154594c524f5352554f5e49555e4f',
     '5a575d525448644b544f5a565448',
     '76525c57525e4f4f5208',
     '785a57525c5a495a7a55555a',
     '5257645949525c5a554f5e0b0c',
     '4858525a554f54505e4858525a',
     '684f544b64685e575d7255514e4942'],
    'opChange': 0.5,
    'orientation': 'Democrat',
    'neighborsOps': [0.5,
     0.75,
     0.8333333333333334,
     0.8703703703703703,
     0.8741830065359477,
     0.8777777777777778,
     0.9022988505747126,
     1.0],
    'error': 0.0015659041394335826,
    'estOp': 0.8317674291938998,
    'openMindedness': 0.5408496732026145,
    'nactiveint': 1},
   '5554555054555d5449565e': {'dataset': 'euro2020',
    'opt': 0.75,
    'opt1': 1.0,
    'neighbors': ['775a7d495a555852090c',
     '5154594c524f5352554f5e49555e4f',
     '5554555054555d5449565e',
     '5257645949525c5a